In [1]:
# pip install git+https://github.com/JustAnotherArchivist/snscrape.git

In [2]:
# # Using TwitterSearchScraper to scrape data and append tweets to list
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('$BTC').get_items()):
#     if i > 300:
#         break
#     tweets_list.append([tweet.date, tweet.content])
# # # Creating a dataframe from the tweets list above
# tweets_df = pd.DataFrame(tweets_list, columns=['created_at', 'full_text'])

In [76]:
# pip install yfinance

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24095 sha256=f8e354513abb3dbe0e61637bc767f075e7aff6e79f1c45dfbc6bc1d65c5c1048
  Stored in directory: /Users/munjikahalah/Library/Caches/pip/wheels/f9/e9/7e/8b13db3bf3aeb5049d759e10702736fb96753089ac950fddc0
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8368 sha256=c4b45a3c13eb8c959a84407fedee5e90c4180c012c7f831be9fcf6cff4d71983
  Stored in directory: /Users/munjikahalah/Library/Caches/pip/wheels/57/6d/a3/a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [77]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import regex as re
import tensorflow_hub as hub
import numpy as np
# import tensorflow as tf
import yfinance as yf
# import tensorflow_text

In [5]:
def replacen(x):
    return x.replace('\n','')
def replacer(x):
    return x.replace('\r','')
def replacern(x):
    return x.replace('\r\n','')

In [6]:
def clean_data(df):
    df['full_text'] = df['full_text'].apply(replacer)
    df['full_text'] = df['full_text'].apply(replacen)
    df['full_text'] = df['full_text'].apply(replacern)
    return df
def remove_dollar(l):
    new_list = []
    for item in l:
        new_list.append(item.replace('$',''))
    return new_list

In [7]:
def remove_urls(text):
    text = re.sub(r'https?://\S+', '', text)
    return text
def remove_url(df):
    df['full_text'] = df['full_text'].apply(remove_urls)
    return df

In [8]:
def find_tickers(text):
    return re.findall(r'[$][A-Za-z][\S]*', text)

In [9]:
# Using TwitterSearchScraper to scrape data and append tweets to list
duplicated_tweet = []
def get_tweets(tickers):
    tweets_list = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('${}'.format(tickers)).get_items()):
        if i > 1000:
            break
        tweets_list.append([tweet.date, tweet.content])
        tweet_df = pd.DataFrame(tweets_list, columns=['created_at', 'full_text'])
        duplicated_tweet.append( tweet_df.sum().duplicated())
        tweet_df['symbols'] = tweet_df['full_text'].apply(find_tickers)
        tweet_df['symbols'] = tweet_df['symbols'].apply(remove_dollar)
        download_yahoo_stocks(tickers= tickers, period='6mo')
        
    return tweet_df

In [10]:
# get_tweets('SOL')

In [11]:
dataframe = get_tweets('DOGE')

/var/folders/fw/j6mc6mcd20379_z864ff2bfw0000gn/T/ipykernel_15395/3011636862.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  duplicated_tweet.append( tweet_df.sum().duplicated())


In [12]:
dataframe = clean_data(dataframe)

In [13]:
def embed_matrix(df):
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
    text_result = embed(df['full_text'])
    vector_df = pd.DataFrame(text_result.numpy())
    return vector_df


In [14]:
dataframe = remove_url(dataframe)

In [15]:
dataframe.full_text[8]

"@BadLuckBanana1 This is not about NFTs, i want to talk about a #Crypto i think is the next big one, if you're not interested remember that you were not interested in $Bit $Eth $Doge $shib years ago and ask to your friends who know about #altcoin to check this out $Tking"

In [16]:
def df_to_datetime(df):
    df.index = pd.to_datetime(df.index)
    return df

In [17]:
def average_tweets_per_day(df):
    ticker_mean_df = df.groupby('created_at').mean().reset_index()
    print(f'TICKER MEAN DF SHAPE =========== {ticker_mean_df.shape}')
    return ticker_mean_df

In [18]:
# def download_yahoo_stocks(tickers=tickers, period = "6mo"):
#     tickers = yf.download(tickers = tickers, interval='1d',period = period)

#     INTCclose = tickers['Close']['INTC']
#     BYNDclose = tickers['Close']['BYND']
#     GEclose = tickers['Close']['GE']
#     BTCclose = tickers['Close']['BTC']

#     # return {"BTC": BTCclose, "BYND": BYNDclose,

#     return INTCclose, BYNDclose, GEclose, BTCclose

In [19]:
def fix_date_time(df):
    if df['created_at'].dtype == 'object':
        df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_at'] = df['created_at'].dt.date
    return df


In [64]:
def grab_live_tweets_to_vectorize(dataframe):
    df = dataframe
    df = fix_date_time(df)
#     df = concat_vectors(df)
    df = embed_matrix(df)
#     df = df_to_datetime(df)
#     df = average_tweets_per_day(df)
#     df = df.set_index(['created_at'])
    
    return df


In [66]:
our_live_vector = grab_live_tweets_to_vectorize(dataframe)

FileNotFoundError: Op type not registered 'SentencepieceOp' in binary running on Munjis-MacBook-Pro.local. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 If trying to load on a different device from the computational device, consider using setting the `experimental_io_device` option on tf.saved_model.LoadOptions to the io_device such as '/job:localhost'.

In [63]:
our_live_vector

,created_at,full_text,symbols
0,2021-11-03,This trader fixed $100thousands on Ethereum sh...,"[eth, omg, ldx, lrc, cate, mana, avax, hot, co..."
1,2021-11-03,@SquidGamePRO @CoinMarketCap @business @nypost...,"[SQUID, Floki, Shiba, Doge, SQUID, Floki, Shib..."
2,2021-11-03,@himacoin I'm so excited and happy to be parti...,"[HIMA, SAMO, SOL, DOGE]"
3,2021-11-03,Bugün patlıycak hissediyorum $DOGE #doge sende...,[DOGE]
4,2021-11-03,@himacoin This project have great potentials a...,"[HIMA, SAMO, SOL, DOGE]"
...,...,...,...
996,2021-11-02,@SquidGamePRO @CoinMarketCap @business @nypost...,"[SQUID, Floki, Shiba, Doge]"
997,2021-11-02,$DOGE | $0.27447 | $0.27484 / $0.27281 / $0.27...,[DOGE]
998,2021-11-02,"SIIM, MEUS QUERIDOS!O @elonmusk também acha qu...","[DOGE., FLOKI]"
999,2021-11-02,@SquidGamePRO @CoinMarketCap @business @nypost...,"[SQUID, Floki, Shiba, Doge]"


In [96]:
def download_yahoo_stocks(tickers= 'SPY SPCE', period = "6mo"):
    tickers = yf.download(tickers = tickers, interval='1d',period = period)

    ticker_price = tickers['Close']['SPY']
    nw = tickers['Close']['SPCE']

    return ticker_pricem, nw

In [97]:
stock_price = download_yahoo_stocks('SPY SPCE', '6mo')

[                       0%                       ]

KeyboardInterrupt: 

In [94]:
INTCclose

Date
2021-05-03    57.259998
2021-05-04    56.900002
2021-05-05    56.849998
2021-05-06    57.189999
2021-05-07    57.669998
                ...    
2021-10-27    47.889999
2021-10-28    48.080002
2021-10-29    49.000000
2021-11-01    49.549999
2021-11-02    49.860001
Name: INTC, Length: 129, dtype: float64